# 3. 보너스 문제
클러스터링된 결과에 대해 Association을 수행하고, 트래킹을 수행하여 **위치, 방위각, 속도**를 추정하라. 사용된 방법론은 보고서에 정리하라. 그리고 추정된 결과를 별도의 파일로 추출/저장하여 제출하라

---
## EKF, UKF를 통해 추정해보기.

In [1]:
import numpy as np
from scipy.io import loadmat

In [34]:
data_name = "20220331T160645_ProjectData.mat"
datas = loadmat(data_name)
# list(datas.keys())

In [35]:
# dict에서 다음과 같이 필요한 값들을 불러온다.
# datas[ax][0][time_stack]
time_stack, = datas['Ax_stack'][0].shape

car_datas = []
for t in range(time_stack):
    vx = datas['Vx_stack'][0][t] 
    ax = datas['Ax_stack'][0][t] 
    ay = datas['Ay_stack'][0][t] 
    sas = datas['SAS_Angle_stack'][0][t] 
    yaw = datas['YAW_RATE_stack'][0][t]
    
    car_data = np.array([vx, ax, ay, sas, yaw], dtype=np.float32)
    car_datas.append(car_data)
car_datas = np.array(car_datas)

### 모델 예측하기

속도, 가속도, 각속도를 반영한 **차량의 모델**을 구축한다. 관련 식을 찾는 것이 중요하다. 위 모델을 구축하면 
A matrix를 구할 수 있다. 이에 따라 공분산 R을 크기를 맞추어 준다. 

이후 예측 과정에서는 로봇의 해당 위치에서 **자코비안 matrix를 구축**한다. 그렇게 H matrix를 만들고 나면, 그에 따른 Q matrix 를 만들어 주면 되겠다.

* input = x, vx, ax, y, vy, ay, theta, v_theta
* z_meas = x, vx, y, vy, theta


(594,)